<h2> Hypertuning using Keras-tuner</h2>

In this notebook, we will try to find the optimal model parameters by using keras-tuner. We tried Ray but we couldn't get that one to work. Not locally but also not in Google Colab.
We will try two hypertunes: randomsearch and hyperband. The best hypertuner will deliver the "winning" model

In [47]:
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sys
from loguru import logger
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.dummy import DummyClassifier
from tensorflow.python.keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping, TensorBoard

import keras_tuner as kt
from keras_tuner.tuners import RandomSearch, Hyperband
from keras_tuner.engine.hyperparameters import HyperParameters as hp
from tensorflow.keras.models import Sequential

sys.path.append('..')

from definitions import get_project_root
from src.data.make_dataset import create_train_test_validation
from src.visualization.visualize import plot_results

root = get_project_root()


In [48]:
tf.random.set_seed(42)

In [49]:
## Create train, validation and test sets
x_train, x_valid, x_test, y_train, y_valid, y_test = create_train_test_validation()
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape

2022-02-12 13:04:28.445 | INFO     | src.data.make_dataset:create_train_test_validation:73 - found file labeled_data.csv, procceed with creating train, test and validation sets


((61711, 23), (61711, 1), (13225, 23), (13225, 1), (13224, 23), (13224, 1))

In [77]:
def model_builder(hp):
    model = Sequential()

    model.add(Dense(23,activation='relu', input_shape=(23,)))

    for i in range(hp.Int("n_layers",1,6)):
        model.add(Dense(hp.Int(f"layer_{i}",5,500,step=50),activation='relu'))

    model.add(Dropout(hp.Float("dropout",0.05,0.4)))
    model.add(Dense(15,activation='softmax'))

    model.compile(optimizer="adam", loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    return model
    

<h2> randomsearch tuner

In [78]:
## Tune the model by using RandomSearch
log_dir = "logs_kt_random"
tensorboard = TensorBoard(log_dir=log_dir)

tuner = RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=15,
    executions_per_trial=3,
    directory=log_dir)

tuner.search_space_summary()

tuner.search(x=x_train,y=y_train,epochs=10,batch_size=64,callbacks=[tensorboard],validation_data=(x_valid,y_valid))

Trial 15 Complete [00h 02m 27s]
val_loss: 1.3695966402689617

Best val_loss So Far: 1.3497042258580525
Total elapsed time: 00h 34m 56s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [79]:
## Store the best hyperparameters as a variable
best_randomsearch_params = tuner.get_best_hyperparameters(num_trials=1)[0]

In [1]:
## Train a model with the best hyperparemeters and save the score of the randomsearch model

randomsearch_model = model_builder(best_randomsearch_params)
history_random = randomsearch_model.fit(x_train, y_train, epochs=50,validation_data=(x_valid,y_valid),callbacks=[EarlyStopping(patience=5,restore_best_weights=True)])

NameError: name 'model_builder' is not defined

In [81]:
randomsearch_result = randomsearch_model.evaluate(x_test,y_test)
randomsearch_result

414/414 [==============================] - 1s 1ms/step - loss: 1.3500 - accuracy: 0.5537


[1.3500087261199951, 0.5536902546882629]

Unfortunately the score of the randomsearch model is lower compared to the base model. Because the randomsearch is in fact searching quite random and we do not have very good resources for hypertuning, we will move on with the hyperband which can be more efficient.

<h3> We can reuse the model_builder function

In [84]:
## Hypertune by using the hyperband tuner

log_dir = "kt_hb_acc2"

tuner_hb = Hyperband(
    model_builder,
    objective="val_loss",
    max_epochs=25,
    factor=3,
    hyperband_iterations=1,
    seed=42,
    directory=log_dir
)

tuner_hb.search(x=x_train,y=y_train,epochs=12,batch_size=64,callbacks=[tensorboard],validation_data=(x_valid,y_valid))

Trial 30 Complete [00h 01m 08s]
val_loss: 1.3541325330734253

Best val_loss So Far: 1.3442531824111938
Total elapsed time: 00h 16m 39s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [89]:
## Test the parameters by training the model with these.
best_hyperband_params = tuner_hb.get_best_hyperparameters(num_trials=1)[0]
hyperband_model = model_builder(best_randomsearch_params)
history_hb = hyperband_model.fit(x_train, y_train, epochs=50,validation_data=(x_valid,y_valid),callbacks=[EarlyStopping(patience=5,restore_best_weights=True)])

TypeError: fit() got an unexpected keyword argument 'maximize'

In [86]:
## Test the model on the test data

hyperband_result = hyperband_model.evaluate(x_test,y_test)


414/414 [==============================] - 1s 1ms/step - loss: 1.3596 - accuracy: 0.5526


In [88]:
file_model = root / 'src' / 'models' / 'winning_hypermodel.model'
randomsearch_model.save(file_model)

INFO:tensorflow:Assets written to: c:\Users\huube\OneDrive\Master of Informatics\Machine Learning\Eindopdracht\src\models\winning_hypermodel.model\assets


INFO:tensorflow:Assets written to: c:\Users\huube\OneDrive\Master of Informatics\Machine Learning\Eindopdracht\src\models\winning_hypermodel.model\assets


<h2> Conclusion </h2>
Unfortunately we ran out of time because I think there is more to gain from the hypertune step. I found this to be the most complex stuff so far. I think there is some optimzation to do in the search space by providing better parameters. It seems quite random that my initial simple model outperforms a hypertuner. But as mentioned, because of time I must move on with the evaluation.